# Импорт функции для загрузки данных из файла  

Функция принимает:  

+ путь до файла (строка)  
+ имя таблицы (строка)  
+ столбцы для выгрузки (кортеж)  
+ столбцы которы преобразуются в числовые данные (кортеж)  
+ столбец по которому объединяются таблицы, данные в столбце должны быть числовые
+ начальная/конечная строка с данными, по умолчанию определяется автоматически (кортеж)  


In [ ]:
import string
import pandas as pd
import os
def load_sheet(path_to_file: str, sheet_name: str, columns_range: tuple, numeric_columns: tuple,\
        merge_by_column: str="A", rows_range: list=None)->pd.DataFrame:
    ''' Load sheet from excel file
    ----------
    Parameters:
    path_to_file : path to excel file (str)
    sheet_name : name of sheet from excel file (str)
    columns_range : tuple with columns headers, max 26 columns (str)
    numeric_columns : tuple with columns headers, that should be numeric (str)
    merge_by_column : column by which sheets will be merged, by default it is first column - "A", column should be numeric
    rows_range : list with start/end numeric data row indexes, by default define range automatically (int)
    -------
    Returns:
    pd_sheet : pandas DataFrame
    '''
    ### Проверяем что файл существует
    if not os.path.exists(path_to_file):
        error_message = f"Не нахожу такого файла: {path_to_file}"
        raise ValueError(error_message)
    ### Формируем заголовок для конечной таблицы
    header = os.path.basename(path_to_file)
    header = header.rsplit(".", maxsplit=1)[0]
    ### Проверяем, что нет повторяющихся столбцов
    if len(set(columns_range)) != len(columns_range):
        error_message = f"Есть повторяющиеся столбцы в {header}"
        raise ValueError(error_message)
    ### Проверяем что столбцы для чисел перечислены в кортеже
    if type(numeric_columns) != tuple:
        error_message = f"Столбцы для чисел должны быть перечислены в кортежах, например так (\"A\",)"
        raise ValueError(error_message)
    ### Загружаем файл
    num_xlsx = pd.ExcelFile(path_to_file)
    pd_sheet = pd.read_excel(num_xlsx, sheet_name, header=None)
    ### Create list with capital letters A,B,C ... J
    col = 26 if len(pd_sheet.columns) > 26 else len(pd_sheet.columns)
    columns = list(string.ascii_uppercase[:col])
    ### Удаляем лишние столбцы
    pd_sheet = pd_sheet.iloc[:, :col]
    pd_sheet.columns = columns
    ### Делаем реиндекс как в Excel
    pd_sheet = pd_sheet.set_index((i+1 for i in pd_sheet.index))
    ### Оставляем только нужные столбцы
    pd_sheet = pd_sheet.loc[:, columns_range]
    ### Конвертируем в числа и строки, не числа становятся NaN
    pd_sheet_num = pd_sheet.apply(lambda x: pd.to_numeric(x.astype(str), errors="coerce", downcast="integer"), axis=1)
    ### Определяем строки с данными в нужных столбцах по столбцу для объединений из numeric_columns
    rows_range = rows_range or [None, None]
    if rows_range[0] is None:
        rows_range[0] = pd_sheet_num[merge_by_column].first_valid_index()
    if rows_range[0] is None:
        error_message = f"Не найдено чисел в столбце {merge_by_column} в файле {header}"
        raise ValueError(error_message)
    if rows_range[1] is None:
        rows_range[1] = pd_sheet_num[merge_by_column].last_valid_index()
    if rows_range[1] is None:
        error_message = f"Не найдено чисел в столбце {merge_by_column} в файле {header}"
        raise ValueError(error_message)
    ### Заполняем NaN оригинальными данными
    pd_sheet = pd_sheet_num.fillna(pd_sheet)
    ### Удаляем лишние строки
    pd_sheet = pd_sheet.loc[rows_range[0]:rows_range[1],]
    ### Устанавливаем мультииндекс
    pd_sheet.columns = pd.MultiIndex.from_product([[header], pd_sheet.columns])
    print(pd_sheet.head(2))
    print("....................")
    print(pd_sheet.tail(2).to_string(header=False))
    return pd_sheet

# Загружаем настройки

Важен порядок, первым должен идти файл с номенклатурой, потом файлы отчетов.  
Порядок и количество должен сохранятся во всех переменных и быть одинаковым.


In [ ]:
### Настройки
pathes_to_files = ("LFLАпрель/LFL_list.xlsx",
                   "LFLАпрель/ИМ/апрель_2021.xlsx",
                   "LFLАпрель/ИМ/апрель_2021.xlsx",
                   "LFLАпрель/ИМ/апрель_2021.xlsx",
                   "LFLАпрель/ИМ/апрель_2021.xlsx",
                   "LFLАпрель/ИМ/апрель_2021.xlsx")
sheet_names = ("TDSheet", "TDSheet", "TDSheet", "TDSheet", "TDSheet", "TDSheet")
columns_range = (("A", "B"),
                 ("A", "H"),
                 ("A", "I"),
                 ("A", "H"),
                 ("A", "I"),
                 ("A", "E", "H", "I"))
# rows_range = ((6, 6835),
#               (10, 3442),
#               (10, 3442),
#               (10, 3442),
#               (10, 3442),
#               (10, 3442))

# Загружаем номенклатуру


In [ ]:
lfl = load_sheet(pathes_to_files[0], sheet_names[0], columns_range[0], ("A",))

# Проверяем на дубликаты в столбце кодов и удаляем их


In [ ]:
duplicated_codes = lfl[lfl.duplicated([lfl.columns[0]], keep=False)]
print("Дубликаты")
print(duplicated_codes)
lfl.drop_duplicates(lfl.columns[0], inplace=True)

# Загружаем наши файлы с отчетами


In [ ]:
reports = []
for i in range(1, len(pathes_to_files)):
    reports.append(load_sheet(pathes_to_files[i], sheet_names[i], columns_range[i], columns_range[i]))

# Проверяем на дубликаты по кодам и объединяем одинаковые


In [ ]:
for report in reports:
    duplicated_codes = report[report.duplicated([report.columns[0]], keep=False)]
    print("Дубликаты в таблице", report.columns[0][0])
    print(duplicated_codes)
    print("..............")
    col = [report.columns[cl] for cl in range(1, len(report.columns))]
    report.loc[duplicated_codes.iloc[0].name,col] = duplicated_codes[col].sum(axis=0)
    report.drop_duplicates([report.columns[0]], inplace=True)

# Присоединить по номенклатуре (ВПР)


In [ ]:
for report in reports:
    result = pd.merge(lfl, report, how="left", left_on=[lfl.columns[0]], right_on=[report.columns[0]])
print(result)